#Parte III - Machine Learning

Entrenar 2 (de tipos distintos, excluyendo regresiones logísticas) modelos (5 puntos cada uno) con búsqueda de hiperparametros (¿cómo conviene elegir los datos de validación respecto de los de train?). Los modelos deben cumplir las siguientes condiciones:
* Deben utilizar top-2 accuracy como métrica de validación.
* Deben medirse solo en validación, no contra test!!!
* Deben ser reproducibles (correr el notebook varias veces no afecta al resultado).
* Deben tener un score en validación superior a 0,3.
* Para el feature engineering debe utilizarse imputación de nulos, mean encoding y one hot encoding al menos una vez cada uno.
* Deben utilizar al menos 40 features (contando cómo features columnas con números, pueden venir varios de la misma variable).
* Deben utilizar CountVectorizer o TfIdfVectorizer para algunos features.
* Deberán contestar la siguiente pregunta: Para el mejor modelo de ambos, ¿cuál es el score en test? (guardar el csv con predicciones para entregarlo después)

# Modelo 2: XGBoost

In [2]:
import pandas as pd

import random

import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
test_set = pd.read_parquet("/content/drive/MyDrive/Organización de Datos/TP3/Music dataset/test.parquet")
train_set = pd.read_parquet("/content/drive/MyDrive/Organización de Datos/TP3/Music dataset/train.parquet")

### Split train y validation

In [5]:
#Hago el split de train y validation por los artistas, con una semilla para hacerlo reproducible.
#El 80% de los artistas va a train y el 20% restante a validation.
#De esta manera, se evalúa al modelo sobre artistas distintos.

artistas = train_set["artist"].unique().tolist()
random.seed(237)
random.shuffle(artistas)
artistas_20 = artistas[:2*(len(artistas)//10)]

validation = train_set[train_set["artist"].isin(artistas_20)]
train = train_set[~train_set["artist"].isin(artistas_20)]

### Imputo valores NaN de algunas columnas

In [ ]:
#OK: popularity, artist, a_songs, a_popularity, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature, valence

#Relleno los lyrics que faltan con un string vacío
train["lyric"].fillna("", inplace=True)
test_set["lyric"].fillna("", inplace=True)
validation["lyric"].fillna("", inplace=True)

#Relleno los idiomas con el idioma más común (inglés)
language_mas_comun = train["language"].value_counts().idxmax()
train["language"].fillna(language_mas_comun, inplace=True)
test_set["language"].fillna(language_mas_comun, inplace=True)
validation["language"].fillna(language_mas_comun, inplace=True)

#Relleno s-label con el valor promedio
s_label_promedio = train["s-label"].mean()
train["s-label"].fillna(s_label_promedio, inplace=True)
test_set["s-label"].fillna(s_label_promedio, inplace=True)
validation["s-label"].fillna(s_label_promedio, inplace=True)

### Mean Encoding de la variable categórica key en función de genre

In [7]:
keys_por_genero = train.groupby(["language", "genre"]).agg(cant_key_genero=('language','count')).reset_index()
cant_por_genero = train.groupby("genre").agg(cant_genero=('genre','count')).reset_index()

proba_key_genero = keys_por_genero.merge(cant_por_genero, on = "genre")
proba_key_genero["proba"] = proba_key_genero["cant_key_genero"] / proba_key_genero["cant_genero"]
proba_key_genero = proba_key_genero[["language", "genre", "proba"]]

proba_key_genero = proba_key_genero.pivot_table(values = "proba", index = "language", columns = "genre", fill_value = 0).add_prefix("key_")

train = train.merge(proba_key_genero, on = "language")
train.drop(["language"], axis=1, inplace=True)

validation = validation.merge(proba_key_genero, on = "language")
validation.drop(["language"], axis=1, inplace=True)

test_set = test_set.merge(proba_key_genero, on = "language")
test_set.drop(["language"], axis=1, inplace=True)

### Separo en x_train e y_train, x_validation e y_validation

In [8]:
#Dropeo las columnas track_name, genre, artist, a_genres, did (leekean el target o no tienen sentido para predecir)

y_test = test_set["genre"]
x_test = test_set.drop(["track_name", "genre", "artist", "a_genres", "did"], axis=1, inplace=False)

y_train = train["genre"]
x_train = train.drop(["track_name", "genre", "artist", "a_genres", "did"], axis=1, inplace=False)

y_validation = validation["genre"]
x_validation = validation.drop(["track_name", "artist", "a_genres", "did"], axis=1, inplace=False)


#Saco los géneros de validation que no están en train (no se pueden predecir cosas para las que no se entrena)

generos = y_train.unique().tolist()
x_validation = x_validation[x_validation["genre"].isin(generos)]
y_validation = y_validation[y_validation.isin(generos)]
x_validation.drop(["genre"], axis=1, inplace=True)

### One Hot Encoding para variables categóricas


In [9]:
#Variables a encodear: key (12), mode (2), time_signature (4)

x_train = x_train.reset_index().drop(["index"], axis=1)
x_validation = x_validation.reset_index().drop(["index"], axis=1)
x_test = x_test.reset_index().drop(["index"], axis=1)

def one_hot_encoding(x_train, x_validation, x_test, feature, **kwargs):
  one_hot_encoder = OneHotEncoder(**kwargs)
  
  encoded_feature_train = pd.DataFrame(one_hot_encoder.fit_transform(x_train[[feature]]).todense().astype(int))
  encoded_feature_valid = pd.DataFrame(one_hot_encoder.transform(x_validation[[feature]]).todense().astype(int))
  encoded_feature_test = pd.DataFrame(one_hot_encoder.transform(x_test[[feature]]).todense().astype(int))
  
  nombres_columnas = one_hot_encoder.get_feature_names_out()
  encoded_feature_train.columns = nombres_columnas
  encoded_feature_valid.columns = nombres_columnas
  encoded_feature_test.columns = nombres_columnas
  
  x_train = x_train.drop([feature], axis=1).join(pd.DataFrame(encoded_feature_train))
  x_validation = x_validation.drop([feature], axis=1).join(pd.DataFrame(encoded_feature_valid))
  x_test = x_test.drop([feature], axis=1).join(pd.DataFrame(encoded_feature_test))
  
  return x_train, x_validation, x_test

x_train, x_validation, x_test = one_hot_encoding(x_train, x_validation, x_test, "key")
x_train, x_validation, x_test = one_hot_encoding(x_train, x_validation, x_test, "mode", drop="first")
x_train, x_validation, x_test = one_hot_encoding(x_train, x_validation, x_test, "time_signature")

### Encoding de lyric con NLP

In [10]:
#Feature: largo del lyric
x_train["lyric_largo"] = x_train["lyric"].apply(lambda x: len(x))
x_validation["lyric_largo"] = x_validation["lyric"].apply(lambda x: len(x))
x_test["lyric_largo"] = x_test["lyric"].apply(lambda x: len(x))

#Feature: cantidad de tokens del lyric
x_train["lyric_cant_tokens"] = x_train["lyric"].apply(lambda x: len(word_tokenize(x)))
x_validation["lyric_cant_tokens"] = x_validation["lyric"].apply(lambda x: len(word_tokenize(x)))
x_test["lyric_cant_tokens"] = x_test["lyric"].apply(lambda x: len(word_tokenize(x)))

#Feature: cantidad de tokens únicos sobre tokens totales
def tokens_unicos_sobre_totales(lyric):
  if len(word_tokenize(lyric)) != 0:
    return len(set(word_tokenize(lyric))) / len(word_tokenize(lyric))
  else:
    return 0
x_train["lyric_tokens_unicos"] = x_train["lyric"].apply(lambda x: tokens_unicos_sobre_totales(x))
x_validation["lyric_tokens_unicos"] = x_validation["lyric"].apply(lambda x: tokens_unicos_sobre_totales(x))
x_test["lyric_tokens_unicos"] = x_test["lyric"].apply(lambda x: tokens_unicos_sobre_totales(x))

#Feature: palabras por minuto
def palabras_por_minuto(lyric, duracion_ms):
  cant_tokens = len(word_tokenize(lyric))
  duracion_min = duracion_ms / 60000
  return cant_tokens / duracion_min
x_train["lyric_pal_por_minuto"] = x_train[["lyric", "duration_ms"]].apply(lambda x: palabras_por_minuto(*x), axis=1)
x_validation["lyric_pal_por_minuto"] = x_validation[["lyric", "duration_ms"]].apply(lambda x: palabras_por_minuto(*x), axis=1)
x_test["lyric_pal_por_minuto"] = x_test[["lyric", "duration_ms"]].apply(lambda x: palabras_por_minuto(*x), axis=1)

#Feature: si el lyric contiene la palabra love
x_train["lyric_tiene_love"] = x_train["lyric"].apply(lambda x: 1 if "love" in x else 0)
x_validation["lyric_tiene_love"] = x_validation["lyric"].apply(lambda x: 1 if "love" in x else 0)
x_test["lyric_tiene_love"] = x_test["lyric"].apply(lambda x: 1 if "love" in x else 0)

#Feature: si el lyric contiene la palabra yeah
x_train["lyric_tiene_yeah"] = x_train["lyric"].apply(lambda x: 1 if "yeah" in x else 0)
x_validation["lyric_tiene_yeah"] = x_validation["lyric"].apply(lambda x: 1 if "yeah" in x else 0)
x_test["lyric_tiene_yeah"] = x_test["lyric"].apply(lambda x: 1 if "yeah" in x else 0)

#Feature: si el lyric contiene la palabra blue/blues
x_train["lyric_tiene_blues"] = x_train["lyric"].apply(lambda x: 1 if ("blue" in x or "blues" in x) else 0)
x_validation["lyric_tiene_blues"] = x_validation["lyric"].apply(lambda x: 1 if ("blue" in x or "blues" in x) else 0)
x_test["lyric_tiene_blues"] = x_test["lyric"].apply(lambda x: 1 if ("blue" in x or "blues" in x) else 0)

TF-IDF

In [23]:
#Al final no terminé utilizando TF-IDF porque empeora la performance del modelo.
#Sólo utilicé Count Vectorizer.

"""stopwords_esp = set(stopwords.words('spanish'))
stopwords_en = set(stopwords.words('english'))
stopwords_en_esp = stopwords_en | stopwords_esp
count_IDF = TfidfVectorizer(lowercase = True, stop_words = stopwords_en_esp, max_features = 10)

matriz_vectores_IDF_train = count_IDF.fit_transform(x_train["lyric"])
df_vectores_IDF_train = pd.DataFrame(matriz_vectores_IDF_train.toarray(), columns = count_IDF.get_feature_names_out())
x_train = x_train.join(df_vectores_IDF_train)

matriz_vectores_IDF_valid = count_IDF.transform(x_validation["lyric"])
df_vectores_IDF_valid = pd.DataFrame(matriz_vectores_IDF_valid.toarray(), columns = count_IDF.get_feature_names_out())
x_validation = x_validation.join(df_vectores_IDF_train)

matriz_vectores_IDF_test = count_IDF.transform(x_test["lyric"])
df_vectores_IDF_test = pd.DataFrame(matriz_vectores_IDF_test.toarray(), columns = count_IDF.get_feature_names_out())
x_test = x_test.join(df_vectores_IDF_train)"""

Count Vectorizer

In [11]:
stopwords_esp = set(stopwords.words('spanish'))
stopwords_en = set(stopwords.words('english'))
stopwords_en_esp = stopwords_en | stopwords_esp

count_vec = CountVectorizer(lowercase = True, stop_words = stopwords_en_esp, max_features = 10)
matriz_vectores_count_vec = count_vec.fit_transform(x_train["lyric"])
palabras_mayor_count = pd.DataFrame(matriz_vectores_count_vec.toarray(), columns = count_vec.get_feature_names_out()).columns.tolist()

#Saco de la lista las palabras love y yeah que ya las había utilizado para hacer features
palabras_mayor_count.remove("love")
palabras_mayor_count.remove("yeah")

for palabra in palabras_mayor_count:
  x_train["lyric_tiene_" + palabra] = x_train["lyric"].apply(lambda x: 1 if palabra in x else 0)
  x_validation["lyric_tiene_" + palabra] = x_validation["lyric"].apply(lambda x: 1 if palabra in x else 0)
  x_test["lyric_tiene_" + palabra] = x_test["lyric"].apply(lambda x: 1 if palabra in x else 0)

In [12]:
#Dropeo la columna lyric una vez obtenidas las features
x_train.drop(["lyric"], axis=1, inplace=True)
x_validation.drop(["lyric"], axis=1, inplace=True)
x_test.drop(["lyric"], axis=1, inplace=True)

### Entrenamiento de un modelo de XGBoost con hiperparámetros default (modelo sin TF-IDF):

In [13]:
modelo = xgb.XGBClassifier(random_state = 237)

modelo.fit(x_train, y_train)

proba_preds = modelo.predict_proba(x_validation)

top_k_accuracy_score(y_validation, proba_preds, k=2, labels=modelo.classes_)

0.5539556962025316



---



### Búsqueda de hiperparámetros con Random Search (modelo con TF-IDF):

In [27]:
modelo = xgb.XGBClassifier(random_state = 237)

hiperparams = {"learning rate": [0.05, 0.1, 0.2], "max_depth": [2, 3, 4, 5], "subsample": [0.5, 0.75, 1], "colsample_bytree": [0.1, 0.25, 0.5, 1], "n_estimators": [50, 100, 150, 200], 
               "objective": ["reg:logistic", "binary:logistic"], "gamma": [0, 0.5, 1, 2], "alpha": [0, 0.5, 1, 2], "lambda": [0, 0.5, 1, 2]}
random_search = RandomizedSearchCV(modelo, hiperparams, n_iter=30, cv=3, random_state=237)
search = random_search.fit(x_train, y_train)

In [28]:
search.best_params_

{'subsample': 1,
 'objective': 'binary:logistic',
 'n_estimators': 200,
 'max_depth': 5,
 'learning rate': 0.2,
 'lambda': 0.5,
 'gamma': 1,
 'colsample_bytree': 0.5,
 'alpha': 2}

Predicciones para valid con el nuevo modelo con los hiperparámetros encontrados:

In [29]:
modelo_mejorado = search.best_estimator_
proba_preds = modelo_mejorado.predict_proba(x_validation)
top_k_accuracy_score(y_validation, proba_preds, k=2, labels=modelo_mejorado.classes_)

0.5314873417721518

### Refino la búsqueda de hiperparámetros:

In [ ]:
modelo = xgb.XGBClassifier(random_state = 237)

hiperparams = {"learning rate": [0.1, 0.2, 0.3], "max_depth": [3, 5, 7], "subsample": [0.50, 1, 1.5], "colsample_bytree": [0.25, 0.5, 0.75, 1], "n_estimators": [100, 200, 300], 
               "gamma": [0, 1, 2], "alpha": [0, 1, 2, 5, 10], "lambda": [0, 0.5, 1]}
random_search = RandomizedSearchCV(modelo, hiperparams, n_iter=30, cv=3, random_state=237)
search = random_search.fit(x_train, y_train)

In [32]:
search.best_params_

{'subsample': 1,
 'n_estimators': 300,
 'max_depth': 7,
 'learning rate': 0.2,
 'lambda': 1,
 'gamma': 0,
 'colsample_bytree': 0.75,
 'alpha': 0}

Predicciones para valid con el nuevo modelo con los hiperparámetros encontrados:

In [33]:
modelo_mejorado = search.best_estimator_
proba_preds = modelo_mejorado.predict_proba(x_validation)
top_k_accuracy_score(y_validation, proba_preds, k=2, labels=modelo_mejorado.classes_)

0.527373417721519

Luego de dos Random Search (uno de 2 horas y otro de 4 horas refinando sobre el anterior) no pude encontrar mejores hiperparámetros que los default. Pruebo sin utilizar el encoding de TF-IDF ya que sospecho que confunde el modelo al obtener un score menor.



---



### Búsqueda de hiperparámetros con Random Search (modelo sin TF-IDF):

In [ ]:
modelo = xgb.XGBClassifier(random_state = 237)

hiperparams = {"learning rate": [0.085, 0.1, 0.115], "max_depth": [5, 7, 12], "subsample": [0.50, 1, 1.5], "colsample_bytree": [0.3, 0.85, 1], "n_estimators": [100, 150, 200], 
               "gamma": [0, 0.05, 0.1], "reg_alpha": [0, 1, 2, 5, 10], "reg_lambda": [0.5, 0.75, 1]}
random_search = RandomizedSearchCV(modelo, hiperparams, n_iter=30, cv=3, random_state=237)
search = random_search.fit(x_train, y_train)

In [56]:
search.best_params_

{'subsample': 1,
 'reg_lambda': 0.75,
 'reg_alpha': 2,
 'n_estimators': 200,
 'max_depth': 12,
 'learning rate': 0.1,
 'gamma': 0.1,
 'colsample_bytree': 1}

Predicciones para valid con el nuevo modelo con los hiperparámetros encontrados:

In [57]:
modelo_mejorado = search.best_estimator_
proba_preds = modelo_mejorado.predict_proba(x_validation)
top_k_accuracy_score(y_validation, proba_preds, k=2, labels=modelo_mejorado.classes_)

0.5316455696202531

### Refino la búsqueda de hiperparámetros:

In [14]:
modelo = xgb.XGBClassifier(random_state = 237)

hiperparams = {"learning rate": [0.090, 0.1, 0.11], "max_depth": [3, 4, 5], "subsample": [0.9, 0.95, 1], "colsample_bytree": [0.9, 0.95, 1], "n_estimators": [95, 100, 105], 
               "gamma": [0, 0.05, 0.1], "reg_alpha": [0, 0.05, 0.1], "reg_lambda": [0.9, 0.95, 1]}
random_search = RandomizedSearchCV(modelo, hiperparams, n_iter=30, cv=3, random_state=237)
search = random_search.fit(x_train, y_train)

In [15]:
search.best_params_

{'subsample': 0.9,
 'reg_lambda': 0.9,
 'reg_alpha': 0.1,
 'n_estimators': 95,
 'max_depth': 5,
 'learning rate': 0.1,
 'gamma': 0,
 'colsample_bytree': 0.9}

Predicciones para valid con el nuevo modelo con los hiperparámetros encontrados:

In [19]:
modelo_mejorado = search.best_estimator_
proba_preds = modelo_mejorado.predict_proba(x_validation)
top_k_accuracy_score(y_validation, proba_preds, k=2, labels=modelo_mejorado.classes_)

0.5509493670886076

### Refino la búsqueda de hiperparámetros:

In [14]:
modelo = xgb.XGBClassifier(random_state = 237)

hiperparams = {"learning rate": [0.1], "max_depth": [5, 6, 7], "subsample": [0.8, 0.9, 1], "colsample_bytree": [0.8, 0.9, 1], "n_estimators": [100, 110, 120], 
               "gamma": [0, 0.1], "reg_alpha": [0, 0.1], "reg_lambda": [0.8, 0.9, 1]}
random_search = RandomizedSearchCV(modelo, hiperparams, n_iter=30, cv=3, random_state=237)
search = random_search.fit(x_train, y_train)

In [23]:
search.best_params_

{'subsample': 0.9,
 'reg_lambda': 1,
 'reg_alpha': 0.1,
 'n_estimators': 120,
 'max_depth': 7,
 'learning rate': 0.1,
 'gamma': 0,
 'colsample_bytree': 1}

Predicciones para valid con el nuevo modelo con los hiperparámetros encontrados:

In [16]:
modelo_mejorado = search.best_estimator_
proba_preds = modelo_mejorado.predict_proba(x_validation)
top_k_accuracy_score(y_validation, proba_preds, k=2, labels=modelo_mejorado.classes_)

0.5382911392405063

In [42]:
modelo = xgb.XGBClassifier(random_state = 237, learning_rate = 0.09, max_depth = 4, subsample = 0.8, colsample_bytree = 0.95,
                           n_estimators = 85, gamma = 0, reg_alpha = 0.15, reg_lambda = 0.9)

modelo.fit(x_train, y_train)

proba_preds = modelo.predict_proba(x_validation)

top_k_accuracy_score(y_validation, proba_preds, k=2, labels=modelo.classes_)

0.5599683544303797

### Refino la búsqueda de hiperparámetros, esta vez me acerco más a los hiperparámetros default que fueron los que mejor resultado me dieron, pero cambiando levemente algunos valores para obtener mejor performance:

In [43]:
modelo = xgb.XGBClassifier(random_state = 237)

hiperparams = {"learning rate": [0.08, 0.09, 0.1], "max_depth": [3, 4], "subsample": [0.8, 0.9, 0.95], "colsample_bytree": [0.95, 1], "n_estimators": [80, 85, 90], 
               "gamma": [0, 0.05], "reg_alpha": [0.1, 0.15], "reg_lambda": [0.85, 0.9]}
random_search = RandomizedSearchCV(modelo, hiperparams, n_iter=25, cv=3, random_state=237)
search = random_search.fit(x_train, y_train)

In [52]:
search.best_params_

{'subsample': 0.95,
 'reg_lambda': 0.9,
 'reg_alpha': 0.1,
 'n_estimators': 90,
 'max_depth': 4,
 'learning rate': 0.08,
 'gamma': 0,
 'colsample_bytree': 0.95}

Predicciones para valid con el nuevo modelo con los hiperparámetros encontrados:

In [45]:
modelo_mejorado = search.best_estimator_
proba_preds = modelo_mejorado.predict_proba(x_validation)
top_k_accuracy_score(y_validation, proba_preds, k=2, labels=modelo_mejorado.classes_)

0.5549050632911392

Este es mi mejor modelo de XGBoost, y dio un score un poco peor para validation que Random Forest (modelo 1), entonces no lo uso para predecir en test